In [1]:
# Modules for data processing
import numpy as np
import os
from netCDF4 import Dataset
import scipy.io as sio
import xlrd

# Modules for plotting
from ipywidgets import *
import matplotlib.pyplot as plt
%matplotlib inline
import bokeh.io
import bokeh.mpl
import bokeh.plotting
bokeh.io.output_notebook()
import seaborn as sns
sns.set_style(style='white')

Loading BokehJS ...

#### Option 1: Load data from an Excel file
<pre>
workbook = xlrd.open_workbook(<span style="color:#cc7a00"><b>'EXCEL_FILE.xlsx'</b></span>)
sheet_names = workbook.sheet_names()
sheet = workbook.sheet_by_name(sheet_names[i])
</pre>

#### Option 2: Load data from a Matlab file
<pre>
matfile  = sio.loadmat(<span style="color:#cc7a00"><b>'MATLAB_FILE.mat'</b></span>)
data = matfile[<span style="color:#cc7a00"><b>'VARIABLE_FORCING'</b></span>]
</pre>

#### Option 3: Load data from a netCDF file
<pre>
nc_fid = Dataset(<span style="color:#cc7a00"><b>'NETCDF_FILE.nc'</b></span>, 'r')
data = nc_fid.variables[<span style="color:#cc7a00"><b>VARIABLE_FORCING</b></span>][:]
</pre>

#### Run the command below to create forcing for Dhara model

In [2]:
# Load matlab file
matfile  = sio.loadmat('flux_data.mat')

# Load variables
doy_in_float = matfile['doy']
doy_in       = doy_in_float.astype(int)
years_in     = matfile['year']
decdoy_in    = matfile['decdoy']
Rg_in        = matfile['Rg_in']
Pa_in        = matfile['Pa_in']
LWdn_in      = matfile['LWdn_in'].T
ZEN_in       = matfile['ZEN_in']
U_in         = matfile['U_in']
PPT_in       = matfile['PPT_in']
Ta_in        = matfile['Ta_in']
ea_in        = matfile['ea_in']
hour_in      = matfile['hour'].T
num_steps_in = len(doy_in)

# Optional - fill in gap for Longwave using Stefan Boltzmann eqn
for i in range(0,num_steps_in):
    if np.isnan(LWdn_in[0][i]):
        LWdn_in[0][i] = 5.670373e-8 * 0.975 * (Ta_in[i]+273.15)**4	
        
# Check/create folder and topography file name
folder = 'data'
filename_output = '/forcings_' + str(num_steps_in) + '.nc'
if not os.path.exists(folder):
    os.makedirs(folder)

################################################################
# DO NOT CHANGE BELOW THIS LINE --------------------------------
################################################################

forcing = Dataset(folder + '/' + filename_output, 'w', format='NETCDF4')
forcing.description = 'Forcing Data for Dhara'

# Set up dimension
forcing.createDimension('time', num_steps_in)	# Time series variable
forcing.createDimension('scalar', 1)			# Scalar variable

"""
Create variables in the netcdf file
	var = netcdf.createVariable('Var_name', 'var_type', ('dimension_type'))
"""
doy       = forcing.createVariable('DayOfYear', 'i4', ('time'))
years     = forcing.createVariable('Year', 'i4', ('time'))
decdoy    = forcing.createVariable('DecimalDayOfYear', 'f8', ('time'))
hour      = forcing.createVariable('Hour', 'f8', ('time'))
Rg        = forcing.createVariable('GlobalRadiation', 'f8', ('time'))
Pa        = forcing.createVariable('AtmosphericPressure', 'f8', ('time'))
LWdn      = forcing.createVariable('LongwaveDownward', 'f8', ('time'))
ZEN       = forcing.createVariable('ZenithAngle', 'f8', ('time'))
U         = forcing.createVariable('WindVelocity', 'f8', ('time'))
PPT       = forcing.createVariable('Precipitation', 'f8', ('time'))
Ta        = forcing.createVariable('AirTemperature', 'f8', ('time'))
ea        = forcing.createVariable('VaporPressureAir', 'f8', ('time'))
num_steps = forcing.createVariable('NumTimeSteps', 'i4', ('scalar'))


# Assign data to variables in NetCDF file
doy[:]       = doy_in
years[:]     = years_in
decdoy[:]    = decdoy_in
Rg[:]        = Rg_in
Pa[:]        = Pa_in
LWdn[:]      = LWdn_in
ZEN[:]       = ZEN_in
U[:]         = U_in
PPT[:]       = PPT_in
Ta[:]        = Ta_in
ea[:]        = ea_in
hour[:]      = hour_in
num_steps[:] = num_steps_in

# Close the file
forcing.close()

#### If you want to visualize forcing data, run the two commands below

In [3]:
def plot_forcing(var_name):
    nc_fid = Dataset(folder + '/' + filename_output, 'r')
    x = nc_fid.variables['DecimalDayOfYear'][:] # common variable on x-axis
    var = nc_fid.variables[var_name][:]
    
    # Plotting data
    fig, ax = plt.subplots(figsize=(9,5))
    ax.plot(x,var)
    ax.set_xlabel('Time (day)')
    ax.set_ylabel(var_name)
    
    # make the plot interactive with Bokeh package
    bokeh.plotting.show(bokeh.mpl.to_bokeh())

In [4]:
var_name = ['AirTemperature',
            'GlobalRadiation',
            'LongwaveDownward',
            'Precipitation',
            'VaporPressureAir',
            'WindVelocity',
            'ZenithAngle',
            'AtmosphericPressure']
interact(plot_forcing, var_name=var_name)

<function __main__.plot_forcing>